In [13]:
import pandas as pd
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
import glob
from dateutil import parser
import re
from geotext import GeoText
import numpy as np
from scipy import stats 
import geopandas as gpd
import datetime

# Spacy for tokenizing our texts

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Gensim is needed for modeling

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Gab Daos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [216]:
# Setting up Spacy Tokenizer
nlp = English()

def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# This will add pipelines in our tokenization process.

nlp.add_pipe(lemmatizer,name='lemmatizer')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [217]:
# This is a function that will create a model that predicts the topics conveyed by each group 

def topic_modeler(tokenized_texts, no_topics, no_words):
    topics = []

    words = corpora.Dictionary(tokenized_texts)
    corpus = [words.doc2bow(doc) for doc in tokenized_texts]

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=words,
                                                random_state = 3,
                                               num_topics= no_topics)
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=words, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return lda_model

In [239]:
files = glob.glob("scraped_data\*.csv")
dfs = [pd.read_csv(f) for f in files]
df = pd.concat(dfs,ignore_index=True)
df['date'] = [parser.parse(date).strftime('%Y-%m-%d') for date in df['date']]
df = df[(df['text'].str.contains('coronavirus'))]
#df = df[df['category'] == 'Philippines']
df = df.drop_duplicates()
df = df.reset_index(drop = True)


location = pd.read_csv('ph_locations.csv')
location = location.applymap(str.lower)

In [175]:
# LDA Topics 

words = df['text'].str.lower()
listWords = []
for item in words:
    listWords.append([nlp(item)])

topics = []
for x in listWords:
    res = topic_modeler(x, 1, 30)
    res = res.show_topic(0, topn = 30)
    topics.append([word[0] for word in res])
    
df['LDA_Topics'] = topics

In [241]:
# Extracting all the counting phrases in the articles

df['count_docs'] =  df['text'].apply(lambda x: re.findall("\d+(?:,\d+)?\s+[a-zA-Z]+", x))

checker = ['confirmed','suspected','quarantine','case','infected','monitoring','chinese','monitored']

count_docs = []
for index, row in df.iterrows():
    passed = []
    for item in row['count_docs']:
        if any(ext in item.lower() for ext in checker):
            passed.append(item)
            break
    
    count_docs.append(passed)

df['count_docs'] = count_docs

In [242]:
# Extracting all the PH Locations using geotext on the articles

df['PH_Loc'] = [list(set(GeoText(content, 'PH').cities)) for content in df['text']]
df['PH_Loc'] = [[x.lower() for x in w] for w in df['PH_Loc']]
df['PH_Loc'] =[[x.replace('city', '') for x in w] for w in df['PH_Loc']]

In [243]:
# Identifying which articles are about suspicious or confirmed cases of the virus
status = []
for index, row in df.iterrows():
    if (('confirmed' in row['LDA_Topics']) | ('confirms' in row['LDA_Topics'])| ('confirm' in row['LDA_Topics'])) & ('confirm' in row['title'])  & (row['date'] >= '2020-01-30'):
        status.append('confirmed')
    elif (('confirmed' in row['LDA_Topics']) | ('confirms' in row['LDA_Topics'])| ('confirm' in row['LDA_Topics']))& (row['date'] >= '2020-01-30'):
        status.append('confirmed')
    elif ('confirm' in row['title']) & ('coronavirus' in row['title']) & (row['date'] >= '2020-01-30'):
        status.append('confirmed')
    elif (any(words in row['LDA_Topics']  for words in ['suspected','quarantine','case','infected','monitoring']))& ('FACT CHECK' not in row['title']) & ('FALSE' not in row['title']):
        status.append('suspected')
    else:
        status.append('')
df['status'] = status

In [244]:
# Selecting Provinces in the identified locations

df['PH_Loc'] = [list(set(loc) & set(location['Pro_Name'].unique())) for loc in df['PH_Loc']]

In [245]:
# For locations not identified through the text, it will check with the LDA topics if a location is identified and use it instead

for index, row in df.iterrows():
    if len(row['PH_Loc']) == 0:
        try:
            df.loc[index, 'PH_Loc'] = [list(set(row['LDA_Topics']) & set(location['Pro_Name'].unique()))]
        except ValueError:
            continue

In [246]:
# Cleaning the document counts to just numbers

counts = []
case = []
for count in df['count_docs']:
    try:
        counts.append(count[0].split(' ')[0])
    except IndexError:
        counts.append(0)
        
    try:
        case.append(count[0].split(' ')[1])
    except IndexError:
        case.append('')

df['counts'] = counts
df['counts'] = [str(count).replace(',', '') for count in df['counts']]
df['counts'] = [str(count).replace('.', '') for count in df['counts']]
df['case'] = case

In [247]:
# Finalizing Locations

ph_loc = []
for loc in df['PH_Loc']:
    try:
        ph_loc.append(loc[0])
    except IndexError:
        ph_loc.append('')
df['Loc'] = ph_loc

In [248]:
# Fixing confirmed counts

count_fixer = []
for index, row in df.iterrows():
    if (row['status'] == 'confirmed') & (row['case'] != 'confirmed'):
        count_fixer.append(1)
    else:
        count_fixer.append(row['counts'])

df['counts'] = count_fixer

In [249]:
df.to_csv('test.csv', index = False)

# Processing for CSV

In [205]:
df = df.reset_index(drop = True)
df.to_csv('abscbn_parsed.csv', index = False)

In [206]:
df = pd.read_csv('abscbn_parsed.csv')

In [207]:
df['source_id'] = 0

In [208]:
df = df.append(pd.read_csv('rappler_parsed.csv'))

In [250]:
df = df[df['counts'] != '1300\n\nConfirmed']
df = df[df['Loc'] != ''] #df[df['Loc'].isin(['cebu', 'aklan', 'tarlac','camiguin'])]


In [251]:
df['counts'] = df.counts.astype(int)

In [252]:
df = df.sort_values('date')

In [253]:
def parse(df):
    #print(df.info())
    
    # Get min/max/mean values
    dfa = pd.pivot_table(df, values = 'counts', index=['date', 'Loc'], columns='status', aggfunc=[min, max, np.mean, stats.mode])
    
    # Remove multi-index
    dfa.columns = ["_".join(pair) for pair in dfa.columns]
    dfa = dfa.reset_index()
    
    # Replace 0 with np.nan to forward fill null values
    dfa = dfa.replace(0, np.nan)
    
    # Forward filling needs to be by area
    places = list(df['Loc'].unique())
    
    global dfb
    dfb = pd.DataFrame()
    for place in places:
        df_temp = dfa[dfa['Loc'] == place].fillna(method='ffill')
        dfb = dfb.append(df_temp)
    return dfb

In [254]:
res = parse(df)

In [255]:
res

,date,Loc,min_,min_confirmed,min_suspected,max_,max_confirmed,max_suspected,mean_,mean_confirmed,mean_suspected,mode_,mode_confirmed,mode_suspected
0,2020-01-21,batangas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"([0], [1])"
2,2020-01-23,cebu,NaN,NaN,NaN,NaN,NaN,555.0,NaN,NaN,185.0,NaN,NaN,"([0], [2])"
1,2020-01-23,aklan,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,"([3], [1])"
3,2020-01-24,cebu,NaN,NaN,14.0,NaN,NaN,881.0,NaN,NaN,447.5,NaN,NaN,"([14], [1])"
5,2020-01-25,cebu,NaN,NaN,14.0,NaN,NaN,881.0,NaN,NaN,447.5,"([0], [1])",NaN,"([14], [1])"
4,2020-01-25,aklan,NaN,NaN,80.0,NaN,NaN,80.0,NaN,NaN,80.0,NaN,NaN,"([80], [1])"
7,2020-01-27,cebu,4.0,NaN,14.0,11.0,NaN,881.0,8.666667,NaN,447.5,"([11], [2])",NaN,"([14], [1])"
6,2020-01-27,camiguin,NaN,NaN,106.0,NaN,NaN,106.0,NaN,NaN,106.0,NaN,NaN,"([106], [1])"
9,2020-01-28,cebu,4.0,NaN,14.0,24.0,NaN,881.0,8.000000,NaN,447.5,"([0], [2])",NaN,"([14], [1])"
8,2020-01-28,aklan,NaN,NaN,11.0,NaN,NaN,11.0,NaN,NaN,11.0,NaN,NaN,"([11], [1])"


In [256]:
res = res[['date','Loc', 'min_suspected','min_confirmed']]

In [257]:
res = res.fillna(0)

In [258]:
res = res.sort_values('date')
res

,date,Loc,min_suspected,min_confirmed
0,2020-01-21,batangas,0.0,0.0
2,2020-01-23,cebu,0.0,0.0
1,2020-01-23,aklan,3.0,0.0
3,2020-01-24,cebu,14.0,0.0
5,2020-01-25,cebu,14.0,0.0
4,2020-01-25,aklan,80.0,0.0
7,2020-01-27,cebu,14.0,0.0
6,2020-01-27,camiguin,106.0,0.0
9,2020-01-28,cebu,14.0,0.0
8,2020-01-28,aklan,11.0,0.0


In [193]:
def add_row(df, row):
    df.loc[-1] = row
    df.index = df.index + 1  
    return df.sort_index()

for Loc in res['Loc']:
    sus_holder = 0
    con_holder = 0
    for date in res['date']:
        if sum(res[res['date'] == date]['Loc'].str.contains(Loc)) > 0:
            
            sus_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_suspected'].iloc[0]
            con_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_confirmed'].iloc[0]
        else:
            add_row(res, [date, Loc, sus_holder, con_holder])
            

In [194]:
res = res.sort_values('date')

In [195]:
res

,date,Loc,min_suspected,min_confirmed
72,2020-01-21,batangas,0.0,0.0
53,2020-01-21,camiguin,0.0,0.0
31,2020-01-21,bohol,0.0,0.0
59,2020-01-21,aklan,0.0,0.0
60,2020-01-21,cebu,0.0,0.0
20,2020-01-21,benguet,0.0,0.0
9,2020-01-21,iloilo,0.0,0.0
42,2020-01-21,laguna,0.0,0.0
71,2020-01-23,batangas,0.0,0.0
30,2020-01-23,bohol,0.0,0.0


In [196]:
prov = gpd.read_file('prov_shp/prov_geo.shp')

In [197]:
df = pd.merge(res, prov, left_on = 'Loc', right_on = 'Pro_Name')

In [198]:
df.head()

,date,Loc,min_suspected,min_confirmed,Pro_Name,centroid,geometry
0,2020-01-21,batangas,0.0,0.0,batangas,"13.891920116590907,121.03136855463036","MULTIPOLYGON (((121.07411 13.53047, 121.07413 ..."
1,2020-01-23,batangas,0.0,0.0,batangas,"13.891920116590907,121.03136855463036","MULTIPOLYGON (((121.07411 13.53047, 121.07413 ..."
2,2020-01-24,batangas,0.0,0.0,batangas,"13.891920116590907,121.03136855463036","MULTIPOLYGON (((121.07411 13.53047, 121.07413 ..."
3,2020-01-25,batangas,0.0,0.0,batangas,"13.891920116590907,121.03136855463036","MULTIPOLYGON (((121.07411 13.53047, 121.07413 ..."
4,2020-01-27,batangas,0.0,0.0,batangas,"13.891920116590907,121.03136855463036","MULTIPOLYGON (((121.07411 13.53047, 121.07413 ..."


In [199]:
df['lat'] = [cor.split(',')[0] for cor in df['centroid']]
df['long'] = [cor.split(',')[1] for cor in df['centroid']]

In [200]:
df = df[['date','Loc','min_suspected','min_confirmed','long','lat']]
df.columns = (['Date','Location','Suspected','Confirmed','Longitude','Latitude'])

In [201]:
df['Date'] = [datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime('%Y-%m-%dT%H:%M:%S.%f') for date in df['Date']]

In [202]:
df = df.sort_values('Date')

In [166]:
df.to_csv('ncov_parsed.csv', index = False)

In [260]:
df[df['counts'] == 881]

,source_id,source_name,date,category,title,author,text,LDA_Topics,count_docs,PH_Loc,status,counts,case,Loc
132,250107.0,rappler,2020-01-24,Rappler IQ,"What we learned from SARS, according to ex-DOH...",Janella Paris,Rappler speaks to former health secretary Manu...,"[dayrit, said, health, sars, virus, doh, china...",[881 cases],"[cebu, iloilo]",suspected,881,cases,cebu
